In [1]:
import numpy as np

import cvxportfolio as cvx

from universes import DOW30, NDX100, SP500

# these are a little more than 500 names, all large cap US stocks
UNIVERSE = sorted(set(DOW30 + NDX100 + SP500))

target_volatility = 0.05 / np.sqrt(252) # annual std

# you can try different risk models (also combining some)
# the full covariance makes this example quite expensive to run
risk_model = cvx.FullCovariance()

constraints = [
    risk_model <= target_volatility**2,
    cvx.DollarNeutral(),
    cvx.MarketNeutral(),
    cvx.LeverageLimit(7),
    cvx.MaxWeights(0.05),
    cvx.MinWeights(-0.05),
]

policy = cvx.SinglePeriodOptimization(
    cvx.ReturnsForecast(),
    constraints = constraints,
    # CVXPY's matrix caching can't be used here, its implementation is
    # only designed to work with small problem instances
    ignore_dpp=True,
    solver='ECOS')

sim = cvx.MarketSimulator(UNIVERSE)

result = sim.backtest(policy, start_time='2000-01-01')

print('RESULT:')
print(result)

figure = result.plot()

Updating data.........................................................

YahooFinance("BBWI") has dubious total open-to-open returns for timestamps: DatetimeIndex(['1982-11-23 14:30:00+00:00', '1985-06-27 13:30:00+00:00'], dtype='datetime64[ns, UTC]', freq=None)


...

YahooFinance("BEN") has dubious total open-to-open returns for timestamps: DatetimeIndex(['1984-11-12 14:30:00+00:00', '1984-11-14 14:30:00+00:00',
               '1984-11-15 14:30:00+00:00', '1984-11-19 14:30:00+00:00',
               '1984-12-28 14:30:00+00:00', '1985-01-02 14:30:00+00:00'],
              dtype='datetime64[ns, UTC]', freq=None)


...

...

.....

YahooFinance("URI") has dubious total open-to-open returns for timestamps: DatetimeIndex(['2007-11-14 14:30:00+00:00'], dtype='datetime64[ns, UTC]', freq=None)


............

YahooFinance("VTRS") has dubious total open-to-open returns for timestamps: DatetimeIndex(['1983-07-01 13:30:00+00:00', '1985-02-19 14:30:00+00:00'], dtype='datetime64[ns, UTC]', freq=None)


............................
